In [0]:
pip install https://github.com/eduardhendriksen/PyForge/archive/master.tar.gz flatten_dict


Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
  Using cached https://github.com/eduardhendriksen/PyForge/archive/master.tar.gz
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached flatten_dict-0.4.2-py2.py3-none-any.whl (9.7 kB)
  Using cached requests_toolbelt-1.0.0-py2.py3-none-any.whl (54 kB)
  Created wheel for PyForge: filename=PyForge-0.4-py3-none-any.whl size=23839 sha256=753d1a5a1c153423926589398e73502a9d0e106b927954ed01fd519288292466
  Stored in directory: /tmp/pip-ephem-wheel-cache-bxjsyqpf/wheels/36/2b/00/cda886582b7b9dcbd63d645cbaed0ae96c3169e5945a72a4f2
Successfully built PyForge
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.


In [0]:
import re
import pandas as pd
import itertools
from flatten_dict import flatten
from time import time

from PyForge import OAuth2Negotiator
from PyForge import FoldersApi
from PyForge import ModelDerivativeApi

In [0]:
def set_log_indent(indent):

     assert indent is not None, "indent is None"
     if isinstance(indent, int):
         indent = "    " * indent
     elif isinstance(indent, float):
         indent = "    " * int(indent)

     def log_indent(message):
         return f"{indent}{message}"

     return log_indent

In [0]:
class ModelData():

    def __init__(self, auth_address, client_id, client_secret, scopes, hub_id, project_name, project_id, folder_id, model_name, view_name):
        self.auth_address = auth_address
        self.client_id = client_id
        self.client_secret = client_secret
        self.scopes = scopes
        self.neg = OAuth2Negotiator(auth_address, client_id, client_secret, scopes)
        self.token, tokenTime = self.neg.get_token()
        self.hub_id = hub_id
        self.project_name = project_name
        self.project_id = project_id
        self.folder_id = folder_id
        self.model_name = model_name
        self.view_name = view_name

        self.model_version = self.get_model_version(self.token, self.project_id, self.folder_id, self.model_name)
        assert self.model_version is not None, "Model Version is None"
        self.model_urn = self.get_model_urn(self.model_version)
        assert self.model_urn is not None, "Model URN is None"
        self.version_number = self.get_version_number(self.model_version)
        assert self.version_number is not None, "Version Number is None"
        
        self.model_object_tree = self.get_model_object_tree_properties(self.token, self.model_urn, self.view_name)  
        assert self.model_object_tree is not None, "Model Oject tree is None" 

        self.model_object_tree_properties = self.get_model_object_tree_properties(self.token, self.model_urn, self.view_name) 
        assert self.model_object_tree_properties is not None, "Model Object tree properties is None" 
            
    def get_model_version(self, token, project_id, folder_id, model_name):
        folders_api = FoldersApi(token)
        f_data, f_included = folders_api.get_folder_contents(project_id=project_id, folder_id=folder_id)
        for thing in f_included:
            if thing['attributes']['name'] == model_name:
                return thing
        return None
    
    def get_model_urn(self, model_version):
        return model_version['id']

    def get_version_number(self, model_version):
        return model_version['attributes']['versionNumber']
    
    def get_model_object_tree_properties(self, token, model_urn, view_name):
        model_derivative_api = ModelDerivativeApi(token)
        model_metadata_ids = model_derivative_api.get_metadata_ids(model_urn)
        for thing in model_metadata_ids['metadata']:
            if thing['name'].lower().replace(' ','') == view_name.lower().replace(' ',''):
                
                model_object_tree = model_derivative_api.get_object_tree(model_urn, thing['guid'])['objects'][0]['objects']
                model_object_properties = model_derivative_api.get_object_properties(model_urn, thing['guid'])['collection']
                return model_object_tree, model_object_properties
            
        return None, None
    def process_object_tree(self, model_object_tree, model_object_properties):

        if isinstance(model_object_tree, list):
            i = 0

            for category in model_object_tree:
                i += 1
                cat_name = category['name']
                print(f"Processing tables for {cat_name}")

                elem_name = cat_name.replace(' ', '').strip()
                type_name = elem_name + 'Types'
                type_frame, elem_frame = self.make_tables(cat_name, model_object_tree, model_object_properties)

                print("Succesfully created tables for {}".format(elem_name))
                

                yield type_name, elem_name, type_frame, elem_frame

            
    def make_tables(self, name, model_object_tree, model_object_properties, logger=None):
        
        msg = set_log_indent(1)
        model_objects = [k for k in model_object_tree if k['name'] == name][0]['objects']
        try:
            model_type_objects = list(itertools.chain.from_iterable([k['objects'] for k in model_objects]))
        except KeyError:
            model_type_objects = None

        model_elems = []
        elem_headers = []
        type_headers = []
        if model_type_objects is not None:

            for type_obj in model_type_objects:

                
                t_0 = time()
                amount = len(type_obj['objects'])
                #print(msg(f"Processing type: {type_obj['name']} with {amount} elements"))
                t = False

                type_obj = self.process_obj(type_obj, model_object_properties, logger=logger, t=t)

                for elem_obj in type_obj['objects']:
                    elem_obj = self.process_obj(elem_obj, model_object_properties, type_obj=type_obj, logger=logger, t=t)
                    elem_obj = flatten(elem_obj, reducer='dot')
                    elem_headers = elem_headers + \
                                    [key for key in elem_obj.keys() \
                                        if key.lower() not in [header.lower() for header in elem_headers]]
                    elem_obj.pop('objects', None)                    
                    model_elems.append(elem_obj)
                    t = False

                type_obj.pop('objects', None)
                type_obj = flatten(type_obj, reducer='dot')
                type_headers = type_headers + \
                                [key for key in type_obj.keys() \
                                    if key.lower() not in [header.lower() for header in type_headers]]

                
                t_exec = "{:.3f}".format(time() - t_0)
                #print(msg(f"Processed  type: {type_obj['name']} with {amount} elements in {t_exec} s"))

            type_frame = pd.DataFrame(model_type_objects, columns=type_headers)
            type_frame.dropna(axis=1, how='all', inplace=True)

        else:

            type_frame = None
            type_obj = None

            
            t_0 = time()
            #print(msg(f"Processing {len(model_objects)} elements"))

            for elem_obj in model_objects:
                elem_obj = self.process_obj(elem_obj, model_object_properties, type_obj=type_obj, logger=logger)
                elem_obj = flatten(elem_obj, reducer='dot')
                elem_headers = elem_headers + \
                                [key for key in elem_obj.keys() \
                                    if key.lower() not in [header.lower() for header in elem_headers]]
                elem_obj.pop('objects', None)                  
                model_elems.append(elem_obj)
               
            
                t_exec = "{:.3f}".format(time() - t_0)
                #print(msg(f"Processed  {len(model_objects)} elements in {t_exec} s"))
        
        elem_frame = pd.DataFrame(model_elems, columns=elem_headers)
        elem_frame.dropna(axis=1, how='all', inplace=True)

        return type_frame, elem_frame

    def process_obj(self, obj, model_object_properties, type_obj=None, logger=None, t=False):
        msg = set_log_indent(2)
        if logger and t:
            t_0_ = time()
        props = [k for k in model_object_properties if k['objectid'] == obj['objectid']][0]
        if logger and t:
            t_exec_0 = "{:.3f}".format(time() - t_0_)
            print(msg(f"Properties found in:        {t_exec_0} s"))
        if logger and t:
            t_0 = time()
        props = flatten(props, reducer='dot')
        if logger and t:
            t_exec_1 = "{:.3f}".format(time() - t_0)
            print(msg(f"Properties flattened in:    {t_exec_1} s"))
        match = None
        if logger and t:
            t_0 = time()

        n_keys = ['index',
                  'Heat Transfer Coefficient',
                  'Thermal Resistance',
                  'Description',
                  'Type Comments',
                  'Type Name',
                  'Rebar Cover']
        keys = [k for k in props.keys() if k not in n_keys]

        for key in keys:
            val = props[key]
            props.update({key : self.convert_to_m(val)})
            if 'Bar Type' in key:
                match = re.match("[ɸ⌀∅Øø](\\d+)\\b", val)
                if match is None:
                    print(msg(f"match is None: \n\t\tproperties.Construction.Bar Type: {val}"))
            if isinstance(val, list):
                props.update({key : str(val)})
            if isinstance(val, tuple):
                props.update({key : str(val)})
            if isinstance(val, dict):
                print(msg(f"Something has gone horribly wrong at this point. Please check the view in BIM360\n\n"))
        if match is not None:
            props.update({'properties.Dimensions.Bar Diameter' : self.convert_to_m(f"{match[1]} mm")})
        obj.update(props)
        if type_obj is not None:
            obj.update({'typeobjectid' : type_obj['objectid']})
        obj.update({'modelVersionNumber' : self.model_version['attributes']['versionNumber']})
        obj.update({'modelVersionCreateTime' : self.model_version['attributes']['createTime']})
        obj.update({'modelName' : self.model_name})
        obj.update({'modelViewName' : self.view_name})
        if logger and t:
            t_exec_2 = "{:.3f}".format(time() - t_0)
            print(msg(f"Properties updated in:      {t_exec_2} s"))
            t_exec = "{:.3f}".format(time() - t_0_)
            #print(msg(f"Total process time:         {t_exec} s"))
        return obj

    def convert_to_m(self, val):
        
        if isinstance(val, str):
            if 'm' in val:
                units = [['', 0],
                         ['d', 1],
                         ['c', 2],
                         ['m', 3]]
                m = 'm'
                dimensions = [[r'\^3', 3],
                              [r'\^2', 2],
                              ['', 1]]
                if val.startswith('[') and val.endswith(']') and ',' in val:
                    val = val[:val.index(',')].strip('[').strip(']').strip("'")
                for u, u_ in units:
                    for d, d_ in dimensions:
                        v = u + m + d
                        match = re.search(f'^(-?\\d*\\.?\\d+)(\\s{v})$', val.strip())
                        if match:
                            v_ = 10**(d_*u_)
                            return float(match[1]) / v_
        return val